In [2]:
import hashlib
import requests
import json
import pymongo
from pymongo import MongoClient
import logging
import threading
agent_account = 'd79bd00d-ca25-4167-96b8-31c4a25778e1'
app_key = '126e7e38-ef8c-427f-8387-90202c9721e4'
api_server = 'http://www.lvcang.cn'
client=MongoClient('mongodb://localhost:27017')
db=client.test
scenic=db.menpiao_scenic_tongcheng
goods=db.menpiao_goods_tongcheng

In [12]:
def increament_post(maxIncreament):
    sign = hashlib.md5(
        '{}{}{}{}'.format(agent_account,maxIncreament, 20, app_key).encode('utf-8')
    ).hexdigest()
    #data
    data={
        "requestHead": {
        "digitalSign": sign,
        "agentAccount": agent_account
        },
        "requestBody": {
        "maxIncrementId": maxIncreament,
        "queryNumber": 20
        }
    }
    #post sce_id景点的第i页
    r=requests.post('http://www.lvcang.cn/jingqu/Services/GetTicketIncrementService.ashx',data=json.dumps(data))
    resp_data = r.json()
    return resp_data.get('maxIncrementId', [])

def increament_for_insert(inc):
    data,maxinc=increament_post(inc)
    if inc!=maxinc:
        thread=threading.Thread(target=increament_for_insert,args=(maxinc,))
        thread.start()
    n=len(data)
    for i_l in range(n):
        item = data[i_l]
        item_dict=item
        for del_key in ['BCTTicketPriceMode','isCopSys','isPreference']:
            item_dict.pop(del_key,'')
        goods.update_one({'ticketPriceId':item_dict['ticketPriceId']},{'$set':item_dict},upsert=True)
        print('>>>更新进度maxIncrement:{}--i:{},启用进程数：{}>>>'.format(inc,i_l,threading.active_count()))


def increament_start():
    try:
        increament_for_insert(0)
    except Exception as e:
        logging.exception(e)

In [15]:
# lis=[]
# def increament_for_insert(inc):
#     data,maxinc=increament_post(inc)
#     if inc!=maxinc:   
#         lis.append(inc)
# def increament_start():
#     try:
#         increament_for_insert(0)
#     except Exception as e:
#         logging.exception(e)
lis=[0]
inc=11670795
maxinc=increament_post(inc)
while inc!=maxinc:        
    lis.append(maxinc)
    print('{}=={}'.format(inc,maxinc) )
    inc=maxinc
    maxinc=increament_post(inc)
print(len(lis))


11670795==11671966
11671966==11672160
11672160==[]
4


In [ ]:
if __name__=='__main__':
    increament_start()